## 02 - Image Enhancement using pixelwise operators

This notebook shows how to enhance images by transforming the pixel values of some input image, into new pixel values producing a better output image

In [ ]:
import numpy as np
import imageio
import matplotlib.pyplot as plt

Pixel-wise operations transform input pixels $z$ that are within some intensity level interval, such as $[0,255]$, into an output pixel $s$.

To illustrate different operators, let us create a vetor with the 256 possible graylevels and work with that to show different types of transformations

In [ ]:
# numpy's arange creates a vetor of a range of values - below, in the interval [0,256)
z = np.arange(256)
print(z)

The `np.arange()` function allows more parameters, in the format `np.arange(start, end, step)`. In such case the functions returns an array whose first value is defined by `start`, the last value is less than `end` and with increment defined by `step`. For example:

In [ ]:
print(np.arange(1, 3, 0.25))

#### Pixel-wise, or point operators
Transform the value $z$ of some input pixel in a new value $s$ via a transformation of the graylevel interval, e.g. 0 to 255. This can be written as: $$s = T(z)$$

For instance, the identity transformation copies the values, without transformation, i.e.:
$$s = T(z) = z$$

The inversion or negative transformation, is given by:
$$s = T(z) = 255-z$$

In [ ]:
s_ident = z
s_inver = 255-z

Because the variable r is actually an array of values (between 0 and 255) we can then plot/visualize the transformation or mapping: one axis shows the $r$ or input values and the other the $s$ or output values:

In [ ]:
plt.figure(figsize=(6,6)) # makes sure the figure will appear with a square size (not rectangular)
plt.plot(z, s_inver) # plots r against s

If we can show the transformation in a plot, we can do the same with actual images.

Let us load images and perform the inversion operation to illustrate it

In [ ]:
img1 = imageio.imread("images/quote2.png")
img2 = imageio.imread("images/scarlett.jpg")

# performing the transformation
img1_invert = 255-img1
img2_invert = 255-img2

In [ ]:
plt.figure(figsize=(12,12)) 
 
# defines a panel to show the images side by side
plt.subplot(121) # panel with 1 row, 2 columns, to show the image at the first (1st) position
plt.imshow(img1, cmap="gray")
plt.axis('off') # remove axis with numbers

plt.subplot(122) # panel with 1 row, 2 columns, to show the image at the second (2nd) position
plt.imshow(img1_invert, cmap="gray")
plt.axis('off')



In [ ]:
plt.figure(figsize=(15,15)) 
 
# defines a panel to show the images side by side
plt.subplot(121) # panel with 1 row, 2 columns, to show the image at the first (1st) position
plt.imshow(img2, cmap="gray")
plt.axis('off') # remove axis with numbers

plt.subplot(122) # panel with 1 row, 2 columns, to show the image at the second (2nd) position
plt.imshow(img2_invert, cmap="gray")
plt.axis('off')

#### Logarithmic enhancement

Mathematical functions are useful to enhance images. 

The $\log$ function in particular decreases the dynamic range between the gray levels so that dark pixels become more bright, while not oversaturating the already bright pixels.

However, it is important to find a log-based transformation to map values between 0 and 255 to values also in the same range. If we just apply a log, for example, using base 2:

$$s = \log_2(z)$$ assuming $$z = 128$$ we would have:

$$s = \log_2(128) = 7$$ which decreased the dynamic range of the gray levels too much and would hamper visualization.

Let us plot this.

In [ ]:
z_log2 = np.log2(z + 1)
plt.figure(figsize=(6,6))
plt.plot(z, z_log2)

We need correction factor to rescale the output into the desired one. This factor is defined as:
$$ \frac{L}{\log_2(1+L)}$$
in which $L$ is the maximum gray levels in the reescaled image, e.g. 255.

In [ ]:
c_scale = 255 / (np.log2(1+255))
z_log2 = c_scale * np.log2(z + 1)
plt.figure(figsize=(6,6)) 
plt.plot(z, z_log2)

Let us apply this in an image now.

In [ ]:
img3 = imageio.imread("images/nap.jpg")
# using the same scale factor, for 255
img3_log2 = (c_scale * np.log2(1 + img3.astype(np.int32))).astype(np.uint8)

In [ ]:
plt.figure(figsize=(15,15)) 
 
plt.subplot(121)
plt.imshow(img3, cmap="gray")
plt.subplot(122)
plt.imshow(img3_log2, cmap="gray")

#### Thresholding

Often applied as a simple segmentation method, it is a step function that outputs only two values, relative to pixels outside or within regions of interest

In [ ]:
# defining some arbitrary threshold T
T = 100

# the where function returns all coordinates matching the conditional
indL = np.where(img1 > T)

# create a new image with the same size of the input 
img1_thresh = np.zeros(img1.shape)

img1_thresh[indL] = 1 # assigns 1 to all pixels above the threshold value

In [ ]:
plt.figure(figsize=(12,12)) 
 
plt.subplot(121)
plt.imshow(img1, cmap="gray")
plt.subplot(122)
plt.imshow(img1_thresh, cmap="gray")

#### Gamma curve correction

Adjust the graylevels using a power transformation. This is useful to adjust the display, so that it better suits the human vision light perception curve

This transformation is given by:
$$s = T(z) = r^{\gamma}$$

Note, again that we need to make sure the output matches the range of graylevels. In this case we are going to first normalize the pixels to 0-1 interval, and then reescale back to 0-255 as follows.

To illustrate, let us also visualize the transformation curve for multiple values of $\gamma$.

In [ ]:
R = 255.00 # maximum graylevel

plt.figure(figsize=(6,6))

# plotting the gamma adjustment function for different Gamma values: between 0.25 and 3.75 with step 0.25
for gamma in np.arange(0.25, 4.0, 0.25):
    z_gamma = (R * np.power(z / R, gamma)).astype(np.uint8)
    plt.plot(z, z_gamma)    

Note that for $\gamma < 1$ it may have effects that is similar to a logarithmic function (increase the dynamic range of dark pixels) as for the orange curve that shows the square root transformation (gamma=0.5), or instead, for $\gamma > 1$, increase the dynamic range of bright pixels.

#### Logistic function
Enhances contrast using sigmoid logistic function, that has an "S" shape.

The parameter k controls the smoothness of the function

In [ ]:
k = 0.025
s_sigmoid = (255/(1+np.exp(-k*(z-127)))).astype(np.uint8)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(z,s_sigmoid)

In [ ]:
k = 0.03
img2_sigm = (255/(1+np.exp(-k*(img2.astype(np.int32)-127)))).astype(np.uint8)

plt.figure(figsize=(15,15)) 
plt.subplot(121)
plt.imshow(img2, cmap="gray")
plt.subplot(122)
plt.imshow(img2_sigm, cmap="gray")

#### Bit plane slicing

We can slice the graylevels in terms of the bits contained in each one of the 8 positions. This may reveal interesting information to better understand where are the most relevant data, and to understand the noise that is often contained in the least significant bits.

In [ ]:
img2 = imageio.imread("images/scarlett_edited.jpg")

plt.figure(figsize=(10,10))

## bit-wise operator AND using mask defined by the bits below
img2_b1 = img2 & 0b10000000
img2_b2 = img2 & 0b01000000
img2_b3 = img2 & 0b00100000
img2_b4 = img2 & 0b00010000
img2_b5 = img2 & 0b00001000
img2_b6 = img2 & 0b00000100
img2_b7 = img2 & 0b00000010
img2_b8 = img2 & 0b00000001

plt.subplot(331)
plt.imshow(img2, cmap='gray')
plt.axis('off')
plt.subplot(332)
plt.imshow(img2_b1, cmap='gray')
plt.axis('off')
plt.subplot(333)
plt.imshow(img2_b2, cmap='gray')
plt.axis('off')
plt.subplot(334)
plt.imshow(img2_b3, cmap='gray')
plt.axis('off')
plt.subplot(335)
plt.imshow(img2_b4, cmap='gray')
plt.axis('off')
plt.subplot(336)
plt.imshow(img2_b5, cmap='gray')
plt.axis('off')
plt.subplot(337)
plt.imshow(img2_b6, cmap='gray')
plt.axis('off')
plt.subplot(338)
plt.imshow(img2_b7, cmap='gray')
plt.axis('off')
plt.subplot(339)
plt.imshow(img2_b8, cmap='gray')
plt.axis('off')